In [3]:
import pandas as pd
import openpyxl
import tkinter as tk
from tkinter import filedialog

# Function to load the Excel files
def load_excel_files():
    root = tk.Tk()
    root.withdraw()
    gstr2b_path = filedialog.askopenfilename(title="Select GSTR-2B Excel File", filetypes=[("Excel Files", "*.xlsx")])
    gstr2a_path = filedialog.askopenfilename(title="Select GSTR-2A Excel File", filetypes=[("Excel Files", "*.xlsx")])
    return gstr2b_path, gstr2a_path

# Function to process and merge the files
def process_and_merge(gstr2b_path, gstr2a_path, output_path):
    # Load the data from both files
    gstr2b = pd.read_excel(gstr2b_path, sheet_name="B2B", header=None, skiprows=6)
    gstr2a = pd.read_excel(gstr2a_path, sheet_name="B2B", header=None, skiprows=6)

    # Define the column headers for GSTR-2B and GSTR-2A
    gstr2b_columns = [
        "GSTIN of supplier", "Trade/Legal name", "Invoice number", "Invoice type",
        "Invoice Date", "Invoice Value(₹)", "Place of supply", "Supply Attract Reverse Charge",
        "Rate(%)", "Taxable Value (₹)", "Integrated Tax(₹)", "Central Tax(₹)",
        "State/UT Tax(₹)", "Cess(₹)", "GSTR-1/5 Period", "GSTR-1/5 Filing Date",
        "ITC Availability", "Reason", "Applicable % of Tax Rate", "Source", "IRN",
        "IRN Date", "Period"
    ]

    gstr2a_columns = [
        "GSTIN of supplier", "Trade/Legal name of the Supplier", "Invoice number", "Invoice type",
        "Invoice Date", "Invoice Value (₹)", "Place of supply", "Supply Attract Reverse Charge",
        "Rate (%)", "Taxable Value (₹)", "Integrated Tax  (₹)", "Central Tax (₹)",
        "State/UT tax (₹)", "Cess  (₹)", "GSTR-1/5 Filing Status", "GSTR-1/5 Filing Date",
        "GSTR-1/5 Filing Period", "GSTR-3B Filing Status", "Amendment made, if any",
        "Tax Period in which Amended", "Effective date of cancellation", "Period"
    ]

    # Assign column headers to the dataframes
    gstr2b.columns = gstr2b_columns
    gstr2a.columns = gstr2a_columns

    # Rename columns for consistent merging
    gstr2a.rename(columns={
        "Trade/Legal name of the Supplier": "Trade/Legal name",
        "Rate (%)": "Rate(%)",
        "Taxable Value (₹)": "Taxable Value (₹)",
        "Integrated Tax  (₹)": "Integrated Tax(₹)",
        "Central Tax (₹)": "Central Tax(₹)",
        "State/UT tax (₹)": "State/UT Tax(₹)",
        "Cess  (₹)": "Cess(₹)"
    }, inplace=True)

    # Merge using "Invoice number" and "GSTIN of supplier"
    merged = pd.merge(
        gstr2b, gstr2a[
            ["Invoice number", "GSTIN of supplier", "Rate(%)", "Taxable Value (₹)", "Integrated Tax(₹)", "Central Tax(₹)", "State/UT Tax(₹)", "Cess(₹)"]
        ],
        on=["Invoice number", "GSTIN of supplier"],
        how="left"
    )

    # Debugging: Print the columns of the merged DataFrame
    print("Columns in merged DataFrame:")
    print(merged.columns.tolist())

    # Remove specified columns from GSTR-2B and GSTR-2A if they exist
    columns_to_remove = ["Rate(%)_x", "Taxable Value (₹)_x", "Integrated Tax(₹)_x", "Central Tax(₹)_x", "State/UT Tax(₹)_x", "Cess(₹)_x"]
    columns_to_remove = [col for col in columns_to_remove if col in merged.columns]
    merged.drop(columns=columns_to_remove, inplace=True)

    # Reorder columns to insert Rate(%)_y, Taxable Value (₹)_y, etc., after "Supply Attract Reverse Charge"
    insertion_columns = ["Rate(%)_y", "Taxable Value (₹)_y", "Integrated Tax(₹)_y", "Central Tax(₹)_y", "State/UT Tax(₹)_y", "Cess(₹)_y"]
    main_columns = merged.columns.tolist()
    position = main_columns.index("Supply Attract Reverse Charge") + 1
    for col in reversed(insertion_columns):
        if col in main_columns:
            main_columns.remove(col)
        main_columns.insert(position, col)

    merged = merged[main_columns]

    # Export the merged file as a table in Excel
    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        merged.to_excel(writer, index=False, sheet_name="Merged Data")
        workbook = writer.book
        worksheet = writer.sheets["Merged Data"]

        # Create an Excel table
        from openpyxl.utils import get_column_letter
        from openpyxl.worksheet.table import Table, TableStyleInfo
        last_column = get_column_letter(merged.shape[1])
        tab = Table(displayName="MergedTable", ref=f"A1:{last_column}{merged.shape[0] + 1}")
        style = TableStyleInfo(
            name="TableStyleMedium9", showFirstColumn=False, showLastColumn=False,
            showRowStripes=True, showColumnStripes=True
        )
        tab.tableStyleInfo = style
        worksheet.add_table(tab)

        # Autofit columns
        for column_cells in worksheet.columns:
            max_length = 0
            column = column_cells[0].column_letter
            for cell in column_cells:
                try:  # Necessary to handle empty cells
                    if cell.value:
                        max_length = max(max_length, len(str(cell.value)))
                except:
                    pass
            adjusted_width = (max_length + 2)
            worksheet.column_dimensions[column].width = adjusted_width

    print(f"Merged file saved as a table to {output_path}")

# Main execution
if __name__ == "__main__":
    print("Please select the GSTR-2B and GSTR-2A files.")
    gstr2b_path, gstr2a_path = load_excel_files()
    output_path = filedialog.asksaveasfilename(
        title="Save Merged File As", defaultextension=".xlsx", filetypes=[("Excel Files", "*.xlsx")]
    )

    if gstr2b_path and gstr2a_path and output_path:
        process_and_merge(gstr2b_path, gstr2a_path, output_path)
    else:
        print("Operation cancelled.")


Please select the GSTR-2B and GSTR-2A files.
Columns in merged DataFrame:
['GSTIN of supplier', 'Trade/Legal name', 'Invoice number', 'Invoice type', 'Invoice Date', 'Invoice Value(₹)', 'Place of supply', 'Supply Attract Reverse Charge', 'Rate(%)_x', 'Taxable Value (₹)_x', 'Integrated Tax(₹)_x', 'Central Tax(₹)_x', 'State/UT Tax(₹)_x', 'Cess(₹)_x', 'GSTR-1/5 Period', 'GSTR-1/5 Filing Date', 'ITC Availability', 'Reason', 'Applicable % of Tax Rate', 'Source', 'IRN', 'IRN Date', 'Period', 'Rate(%)_y', 'Taxable Value (₹)_y', 'Integrated Tax(₹)_y', 'Central Tax(₹)_y', 'State/UT Tax(₹)_y', 'Cess(₹)_y']
Merged file saved as a table to D:/OneDrive/Office-One Drive/Client Excel/Adinn Reco/Python Output.xlsx
